In [17]:
from navitia_client import Client
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon

import pandas as pd
import numpy as np
import copy
import json

pd.set_option('display.max_columns', 999)
pd.set_option('display.max_colwidth', 10000)

# give the TravelMyWay API jey
client = Client(user='8ad3db27-5eec-473d-9ff6-50d35fdf0da6')


In [4]:
def get_navitia_coverage():
    """
    The navitia API is separated into different coverage region (one for california, one for PAris-IDF ...)
    We call the API to get all those coverage and know which coverage region to call for a given scenario
    """
    # call API for all coverages
    response_cov = client.raw('coverage', multipage=False, page_limit=10, verbose=True)
    # turn coverage into DF
    df_cov = pd.DataFrame.from_dict(response_cov.json()['regions'])
    
    print(f'{df_cov.shape[0]} regions found, here is an example:\n {df_cov.sample()}')
    # clean the geographical shape
    df_cov['polygon_clean'] = df_cov.apply(clean_polygon_for_coverage, axis=1)
    return df_cov


def clean_polygon_for_coverage(x):
    """
    The API call for coverage returns multipolygons (a list of one or several polygon) for each region
    but it is a string that we must convert to an actual Polygon object (in order to use function is point in polygon)
    Most regions have only one polygon, so we decide to only consider the biggest polygon for each regions 
    """
    # split "polygon" as a string
    if x['shape'] == '':
        # Polygon is null
        return None
    
    # split by '(' to see if there are several shape within polygon
    split_meta = x['shape'].split('(')
    # we want ton only keep the biggest Polygon, first we compute sizes for all "polygon"
    sizes_pol = np.array([])
    for i in split_meta:
        sizes_pol = np.append(sizes_pol,len(sizes_pol))
    # keep the biggest and act like there was only one from the beginning
    split_pol = split_meta[np.argmax(sizes_pol)]
    
    # Let's split the polygon into a list of geoloc (lat, long)
    split_pol = split_pol.split(',')
    # clean the last point (the first and the last are the same cause the polygon has to be "closed")
    split_pol[-1] = split_pol[0]
    # recreate latitude and longitude list
    lat = np.array([])
    long =  np.array([])
    for point in split_pol:
        split_point = point.split(' ')
        lat = np.append(lat,split_point[0])
        long = np.append(long,split_point[1])
    
    # return the object Polygon
    return Polygon(np.column_stack((long, lat)))


def find_navita_coverage_for_points(point_from, point_to, df_cov):
    """
    This function finds in which coverage regions are the 2 points. 
    If any point is not in any region, or the 2 points are in different regions we have an error
    """
    # test if points are within polygon for each region
    are_points_in_cov = df_cov[~pd.isna(df_cov.polygon_clean)].apply(lambda x: (x.polygon_clean.contains(point_from))&(x.polygon_clean.contains(point_to)), axis=1)
    # find the good region 
    id_cov = df_cov[~pd.isna(df_cov.polygon_clean)][are_points_in_cov].id
    if not id_cov.empty:
        return id_cov.values[0]
    else:
        return 'no region found'


In [5]:
def section_json_summary(x):
    """
    For each section we create a json with type of transportation + duration
    """
    json_summary = {}
    if not pd.isna(x['display_informations']):
        json_summary["type"] = x['display_informations']['physical_mode']
    else :
        json_summary["type"] = x['type']
    json_summary["duration"] = x['duration']

    return json_summary

def get_section_details(x):
    """ 
    For each journey we list all the section summary (type + duration)
    """
    sections = pd.DataFrame.from_dict(x.sections)
    sections['summary'] = sections.apply(section_json_summary, axis=1)
    section_details = []
    for value in sections['summary']: 
        section_details.append(value)
    return section_details
                   
                   
def compute_journey(point_from,point_to):
    """
    Main function takes two points and compute detailed journeys between the 2. It does all the necessary steps
    1 get the relevant coverage region
    2 Call the navitia API
    3 Compute details for each journey
    """
    coverage_region = find_navita_coverage_for_points(point_from,point_to , df_cov)
    if coverage_region == 'no region found':
        return 'the points are not within the voverage of navitia API'
    url = f'coverage/{coverage_region}/journeys?from={point_from.y};{point_from.x}&to={point_to.y};{point_to.x}'
    response = client.raw(url, multipage=False, page_limit=10, verbose=True)
    
    df_journey = pd.DataFrame.from_dict(response.json()['journeys'])
    print(f'{df_journey.shape[0]} journeys found')
    df_journey['section_details'] = df_journey.apply(get_section_details, axis=1)
    df_journey['price_total'] = df_journey.apply(lambda x: x.fare['total']['value'], axis = 1)
    return df_journey[[]]


In [8]:
df_cov = get_navitia_coverage()

Import on url coverage 
45 regions found, here is an example:
    dataset_created_at end_production_date  id     last_load_at    name                                              shape start_production_date   status
36    20191002T084656            20191214  se  20191011T003103  Sweden  MULTIPOLYGON(((20.99723 68.90611,21.37536 68.7...              20191004  running


In [19]:
journeys.columns

Index(['arrival_date_time', 'calendars', 'co2_emission', 'departure_date_time', 'distances', 'duration', 'durations', 'fare', 'links', 'nb_transfers', 'requested_date_time', 'sections', 'status', 'tags', 'type', 'section_details', 'price_total'], dtype='object')

In [20]:
"""
To test the API, you can enter any point from and to and run the compute_journey function
"""

latitude_from = 48.88471
longitude_from = 2.370697
latitude_to = 48.78471
longitude_to = 2.470697

journeys = compute_journey(Point(latitude_from,longitude_from),Point(latitude_to,longitude_to))
journeys[['arrival_date_time', 'calendars', 'co2_emission', 'departure_date_time', 'distances', 'duration',
          'durations', 'fare', 'nb_transfers',
          'requested_date_time', 'status', 'tags', 'type', 'section_details', 'price_total']]

Import on url coverage/fr-idf/journeys?from=2.370697;48.88471&to=2.470697;48.78471 
2 journeys found


,arrival_date_time,calendars,co2_emission,departure_date_time,distances,duration,durations,fare,nb_transfers,requested_date_time,status,tags,type,section_details,price_total
0,20191012T192931,"[{'active_periods': [{'begin': '20191012', 'end': '20191020'}], 'week_pattern': {'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': True}}]","{'value': 491.5655, 'unit': 'gEC'}",20191012T182654,"{'taxi': 0, 'car': 0, 'walking': 376, 'bike': 0, 'ridesharing': 0}",3757,"{'taxi': 0, 'walking': 925, 'car': 0, 'ridesharing': 0, 'bike': 0, 'total': 3757}","{'found': True, 'total': {'currency': 'centime', 'value': '380.0'}, 'links': [{'internal': True, 'type': 'ticket', 'id': 'ticket_1_0', 'rel': 'tickets', 'templated': False}, {'internal': True, 'type': 'ticket', 'id': 'ticket_2_0', 'rel': 'tickets', 'templated': False}]}",2,20191012T181911,OTHER_EFFECT,"[walking, ecologic]",best,"[{'type': 'street_network', 'duration': 246}, {'type': 'Métro', 'duration': 420}, {'type': 'transfer', 'duration': 252}, {'type': 'waiting', 'duration': 228}, {'type': 'Métro', 'duration': 1260}, {'type': 'transfer', 'duration': 336}, {'type': 'waiting', 'duration': 264}, {'type': 'Bus', 'duration': 660}, {'type': 'street_network', 'duration': 91}]",380.0
1,20191012T193112,"[{'active_periods': [{'begin': '20191012', 'end': '20191027'}], 'week_pattern': {'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': True}}]","{'value': 46.794, 'unit': 'gEC'}",20191012T182354,"{'taxi': 0, 'car': 0, 'walking': 1632, 'bike': 0, 'ridesharing': 0}",4038,"{'taxi': 0, 'walking': 1710, 'car': 0, 'ridesharing': 0, 'bike': 0, 'total': 4038}","{'found': True, 'total': {'currency': 'centime', 'value': '190.0'}, 'links': [{'internal': True, 'type': 'ticket', 'id': 'ticket_3_0', 'rel': 'tickets', 'templated': False}]}",1,20191012T181911,OTHER_EFFECT,"[walking, ecologic]",comfort,"[{'type': 'street_network', 'duration': 246}, {'type': 'Métro', 'duration': 420}, {'type': 'transfer', 'duration': 252}, {'type': 'waiting', 'duration': 168}, {'type': 'Métro', 'duration': 1740}, {'type': 'street_network', 'duration': 1212}]",190.0


In [14]:
# See the details of a given journey
journeys.section_details[0]

[{'type': 'street_network', 'duration': 246},
 {'type': 'Métro', 'duration': 180},
 {'type': 'transfer', 'duration': 210},
 {'type': 'waiting', 'duration': 210},
 {'type': 'Train de banlieue / RER', 'duration': 900},
 {'type': 'transfer', 'duration': 336},
 {'type': 'waiting', 'duration': 264},
 {'type': 'Bus', 'duration': 360},
 {'type': 'waiting', 'duration': 480},
 {'type': 'Bus', 'duration': 600},
 {'type': 'street_network', 'duration': 91}]